In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data = pd.read_table('/content/drive/MyDrive/bigleader/shopping.txt')

In [ ]:
data.columns = ['score', 'content']

In [ ]:
# data

In [ ]:
data['score']=data['score'].replace([1,2,4,5],[0,0,1,1])

In [ ]:
# data

In [ ]:
data.info()
data.describe()
data.score.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   score    199999 non-null  int64 
 1   content  199999 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


0    100037
1     99962
Name: score, dtype: int64

In [ ]:
data['content'].nunique()

199907

In [ ]:
data.drop_duplicates(subset=['content'], inplace=True)

In [ ]:
len(data['content'])

199907

In [ ]:
ydata=data['score']
xdata=data['content']

In [ ]:
# !pip install konlpy

In [ ]:
from konlpy.tag import Okt

okt=Okt()

stop_words = [
    '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가',
    '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저',
    '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '그런데', '하지만', '만일', '위하여',
    '하여야', '비하면', '시키다', '하게하다', '하게 되다', '놓다', '있다', '그렇다', '크다', '작다', '같다', '말하다', '생각하다',
    '느끼다', '그러나', '그런', '한', '다른', '다른', '더러', '잠깐', '하다가', '이렇게', '그때', '이런', '저런', '따위', '것',
    '것들', '무엇', '무슨', '어느', '어떤', '언제', '어디', '어느', '어떻게', '얼마', '여기', '기점으로', '기점에서', '빨리',
    '거의', '하물며', '이래', '그래', '다만', '만약', '하니', '정도', '하먼', '힘입어', '험차', '남짓', '할지라도', '말할것도 없고',
    '할망정', '하면', '할수록', '주룩주룩', '지든지', '할지언정', '할지라도', '지든지', '더군다나', '거창', '만 못하다', '할 따름이다',
    '말할것도 없고', '무릎쓰고', '개의치 않고', '하는 김에', '겁쟁이를', '깨고', '부르짖고', '놀래키고', '힘입어', '험차', '남짓',
    '눈을 감고', '더욱더', '불구하고', '얼마든지', '마음대로', '주저하지 않고', '당장', '하자마자', '바로', '당장', '하면된다',
    '즉시', '바로', '하면', '될것이다', '그래', '그렇지', '요컨대', '다시 말하자면', '바꿔 말하면', '즉', '구체적으로', '말하자면',
    '시작하여', '시초에', '이상', '과 같은', '관계가 있다', '관련이 있다', '연관되다', '어떤', '에 대해', '이러한', '에 따르면',
    '에 있다', '이것', '이것들', '여러분','이렇게', '전혀', '모두', '아무',
    '어느', '또한', '심지어', '조차', '한켠으로는', '그중에서', '본대로', '자', '이', '여', '여야', '여도', '하도록시키다', '하기 위하여',
    '위해서', '때문에', '아야', '하곤하였다', '다른', '다른', '뛰어난', '특별한', '얼마', '하느니', '하면 할수록', '일', '하면서',
    '자마자', '하자마자', '둘째, 더욱더', '더욱이는', '뿐만 아니라', '그렇지 않으면', '그렇지 않다면', '그렇지', '않으면', '안', '그러나',
    '그렇지만', '이렇지만', '아니면', '반대로', '말하자면', '바꾸어서 말하면', '하나', '이', '일', '허', '하', '바', '바',
    '제', '가', '은', '는', '와', '과', '도', '의', '에게', '에게서', '뿐이다', '에게서', '으로서', '보는',
    '것', '과의', '보다', '된', '의해', '할', '것이다', '같다', '했다', '해야한다', '한다', '해야한다', '되는', '된다'
]

tok_data=[]

for content in xdata :
  tok_sent=okt.morphs(content)
  sw_rem_sent=[w for w in tok_sent if not w in stop_words]
  tok_data.append(sw_rem_sent)

print(tok_data[0], tok_data[1], tok_data[2], tok_data[3], tok_data[4])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
! --NotebookApp.iopub_data_rate_limit=1000000000

/bin/bash: line 1: --NotebookApp.iopub_data_rate_limit=1000000000: command not found


In [ ]:
#층화추출:종류에따라 균등하게 분리하여 추출
xtrain, xtest, ytrain, ytest=train_test_split(tok_data,ydata,test_size=0.2,
                                              random_state=902,
                                              stratify=ydata)

In [ ]:
tok=Tokenizer()
tok.fit_on_texts(xtrain) #xtrain를 코퍼스로하여 fitting(단어별로 index가 부여됨)
xtrain_enc=tok.texts_to_sequences(xtrain)

tok.index_word
w2i=tok.word_index

type(xtrain_enc)
xtrain_enc
sum(map(len, xtrain_enc))/len(xtrain_enc) #메일의 평균 단어 갯수

max(map(len, xtrain_enc)) #메일의 최대 단어 갯수
min(map(len, xtrain_enc))#메일의 최대 단어 갯수

maxLen=max(len(s) for s in xtrain_enc)

xtrain_padded=pad_sequences(xtrain_enc, maxlen=maxLen)

vocab_size=len(w2i)+1

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

history=model.fit(xtrain_padded, ytrain, epochs=5, batch_size=32, validation_split=0.2)

xtest_enc=tok.texts_to_sequences(xtest)
xtest_padded=pad_sequences(xtest_enc, maxlen=maxLen)
#model.predict(xtest_padded)
model.evaluate(xtest_padded,ytest)
model.evaluate(xtest_padded,ytest)[1]

Epoch 1/5
3999/3999 [==============================] - 207s 50ms/step - loss: 0.3212 - accuracy: 0.8715 - val_loss: 0.2721 - val_accuracy: 0.8977
Epoch 2/5
3999/3999 [==============================] - 194s 48ms/step - loss: 0.2927 - accuracy: 0.8880 - val_loss: 0.2883 - val_accuracy: 0.8918
Epoch 3/5
3999/3999 [==============================] - 194s 48ms/step - loss: 0.2945 - accuracy: 0.8878 - val_loss: 0.3081 - val_accuracy: 0.8835
Epoch 4/5
3999/3999 [==============================] - 204s 51ms/step - loss: 0.2740 - accuracy: 0.8976 - val_loss: 0.3523 - val_accuracy: 0.8498
Epoch 5/5
1250/1250 [==============================] - 10s 8ms/step - loss: 0.2570 - accuracy: 0.9073


0.9072582721710205